In [ ]:
# %pip install langchain faiss-cpu beautifulsoup4 sentence-transformers langchain_community unstructured spacy

In [ ]:
# python -m spacy download en_core_web_sm #in terminal

C:\Users\SHRIVED\AppData\Local\Programs\Python\Python312\python.exe: No module named spacy


In [16]:
from langchain.document_loaders import UnstructuredURLLoader

urls = ["https://brainlox.com/courses/category/technical"]
loader = UnstructuredURLLoader(urls=urls)
docs = loader.load()
# print(docs[0].page_content)

In [17]:
type(docs[0].page_content)

str

In [24]:
%pip install spacy

  Using cached spacy-3.8.4-cp312-cp312-win_amd64.whl.metadata (27 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.12-cp312-cp312-win_amd64.whl.metadata (2.2 kB)
  Using cached cymem-2.0.11-cp312-cp312-win_amd64.whl.metadata (8.8 kB)
  Using cached preshed-3.0.9-cp312-cp312-win_amd64.whl.metadata (2.2 kB)
  Using cached thinc-8.3.4-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached srsly-2.5.1-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached typer-0.15.1-py3-none-any.whl.metadata (15 kB)
  Using cached langcodes-3.5.0-py3-none-any.whl.metadata (29 kB)
  Using cached language_data-1.3.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached blis-1.2.0-cp312-cp312


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: d:\AI_Internship_Task\env\Scripts\python.exe -m pip install --upgrade pip


In [18]:
# from langchain.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://brainlox.com/courses/category/technical")
# docs = loader.load()
# print(docs[0].page_content)

In [19]:
# type(docs[0])

In [22]:
# import spacy
# import tiktoken
# import regex as re

# nlp = spacy.load("en_core_web_sm")

# doc = nlp(docs[0].page_content.lower())
# lemmas = [
#     token.lemma_ for token in doc
#     if not token.is_punct and not token.is_space and not token.is_stop
# ]

# clean_text = " ".join(lemmas)
# clean_text = re.sub(r"[^a-zA-Z0-9\s]", " ", clean_text)

# enc = tiktoken.get_encoding("cl100k_base")
# tokens = enc.encode(clean_text)

# print(lemmas)
# print(tokens)


In [35]:
from sentence_transformers import SentenceTransformer
import spacy

model = SentenceTransformer('all-MiniLM-L6-v2')

nlp = spacy.load("en_core_web_sm")

In [36]:
doc = nlp(docs[0].page_content)
s = [sent.text.split('\n\n') for sent in doc.sents]
sentences = []
for x in s:
  sentences.extend(x)

embedding = model.encode(sentences)

print("Embedding Vector:", embedding)

Embedding Vector: [[ 0.00379    -0.03536956 -0.01876178 ... -0.02447501  0.02312467
  -0.00917474]
 [-0.06056062 -0.00900881  0.02470006 ...  0.01727293  0.05133026
  -0.02029377]
 [-0.0106903   0.06531819  0.05843372 ... -0.03404168  0.04074111
  -0.04138395]
 ...
 [-0.08517705  0.04589946  0.03972438 ... -0.04475846 -0.03499537
  -0.0209472 ]
 [-0.0406027   0.07318036 -0.00524363 ...  0.02927013 -0.04795929
  -0.01991415]
 [-0.05609618  0.08846758  0.02811747 ...  0.01351376  0.06349049
   0.00832166]]


In [37]:
import numpy as np
import faiss

embeddings_np = np.array(embedding).astype('float32')

dim = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dim)

index.add(embeddings_np)

print(f"Number of vectors in the index: {index.ntotal}")

Number of vectors in the index: 424


In [38]:
import pickle

# with open("sentences.pkl", "wb") as f:
#     pickle.dump(sentences, f)

In [39]:
with open("sentences.pkl", "rb") as f:
    sentences = pickle.load(f)

In [40]:
import faiss

# faiss.write_index(index, "my_index.faiss")

In [41]:
index = faiss.read_index("my_index.faiss")

In [42]:
def get_response(query, k):
  query_embedding = model.encode([query]).astype('float32')
  D, I = index.search(query_embedding, k)
  return [{'sentence': sentences[I[0][i]], 'distance': D[0][i]} for i in range(k)]

In [44]:
print(get_response("machine learning", 3))

[{'sentence': 'Kids will tackle machine learning', 'distance': 0.9251971}, {'sentence': 'Machine Learning: 7-Day Project-Based Summer Camp', 'distance': 0.9358075}, {'sentence': 'Join our "Machine Learning Mania" camp for a 7-day tech adventure!', 'distance': 1.1167803}]
